In [0]:
gold=spark.table('ecommerce.gold.events')


In [0]:
gold.columns

In [0]:
%sql
WITH daily AS (
  SELECT
    event_date,
    SUM(revenue) AS rev
  FROM ecommerce.gold.events
  GROUP BY event_date
)
SELECT
  event_date,
  rev,
  AVG(rev) OVER (
    ORDER BY event_date
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) AS ma7
FROM daily
ORDER BY event_date;


In [0]:
%sql
-- Conversion funnel
SELECT category_code,
  SUM(views) as views,
  SUM(purchases) as purchases,
  ROUND(SUM(purchases)*100.0/SUM(views), 2) as conversion_rate
FROM ecommerce.gold.events
GROUP BY category_code;

In [0]:
%sql
-- Customer tiers
SELECT
  CASE WHEN cnt >= 10 THEN 'VIP'
       WHEN cnt >= 5 THEN 'Loyal'
       ELSE 'Regular' END as tier,
  COUNT(*) as customers,
  AVG(total_spent) as avg_ltv
FROM (SELECT user_id, COUNT(*) cnt, SUM(price) total_spent
      FROM ecommerce.silver.events WHERE event_type='purchase' GROUP BY user_id)
GROUP BY tier;
